---
# **機械学習特論　第12回課題**
## **【word2vec】単語のベクトル化および単語間の演算**
---

In [18]:
# ライブラリの読み込み
# %pip install --upgrade pip
# %pip install numpy
# %pip install matplotlib
# %pip install pandas
# %pip install openpyxl
# %pip install sympy
# %pip install scipy
# %pip install re
# %pip install jaconv
# %pip install scikit-learn
# %pip install statsmodels
# %pip install seaborn
# %pip install pmdarima
# %pip install kneed
# %pip install scikit-learn-intelex
# %pip install lightgbm
# %pip install opencv-python
# %pip install ipywidgets
# %pip install gensim

import cv2
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider

from matplotlib.font_manager import FontProperties
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import random
import sympy as sp
import scipy.stats as stats
import re
# import jaconv
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import itertools
import matplotlib.dates as mdates
# import pmdarima as pm
from IPython.display import clear_output
from sklearn.metrics import r2_score, f1_score, mean_absolute_error, mean_squared_error
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
# from kneed import KneeLocator
from sklearn.decomposition import PCA
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import time
from matplotlib.ticker import LogLocator, LogFormatter
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn import metrics
from sklearn.datasets import load_iris
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.exceptions import ConvergenceWarning
# from sklearnex import patch_sklearn
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
import os
# import tensorflow as tf
from sklearn import preprocessing
# from tensorflow.keras.layers import Conv1D, Conv2D, Flatten, Dense, MaxPooling2D, MaxPooling1D
import cv2
from ipywidgets import interact, FloatSlider, IntSlider
import gensim
import gensim.downloader as api

# print(tf.__version__)
# print(tf.config.list_physical_devices('GPU'))

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='seaborn')
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning, message=".*dask-expr is not installed.*")
warnings.filterwarnings("ignore", category=FutureWarning, module="dask.dataframe")


### 1. 機械学習による単語のベクトル化
　英単語に関しては，GloVe (Global Vectors for Word Representation) というWikipedia 2014と Gigaword 5のテキストデータから学習された英語の単語埋め込みモデルを使用する．単語ベクトルは50次元で表現されており，軽量でありながら高い精度を持つのが特徴である．
- ケース感度：非ケースセンシティブ（大文字・小文字を区別しない）
- 用途：類似度計算，意味的クラスタリング，情報検索，NLPモデルの初期埋め込みベクトルとして使用可能
- 利点：小さい次元数でも，単語の意味的類似性を捕捉可能
- 制約：英語に特化しているため，多言語や専門用語には向かない場合あり

　日本語に関しては，chiveという主にニュース記事や日本語Webページのデータセットを元に学習された日本語特化の単語埋め込みモデルを使用する．単語ベクトルは90次元で表現されており，日本語特有の語彙や文脈を考慮し，語彙の意味関係を学習するよう設計されている．
- ケース感度：日本語特有の漢字，ひらがな，カタカナ，アルファベットなどの表記揺れに対応
- 用途：日本語テキストにおける類似度計算，トピックモデリング，日本語NLPタスクのベースモデル
- 利点：日本語に特化した設計で，日本語文脈に基づく高い意味的精度を提供
- 制約：日本語以外の言語では利用不可＆日本語固有の特殊な語彙や専門用語については十分にカバーされていない可能性あり

In [19]:
wv = gensim.downloader.load('glove-wiki-gigaword-50') # Wikipedia 2014 + Gigaword 5 (6B tokens, uncased)
wv_jp = gensim.models.KeyedVectors.load('chive-1.2-mc90_gensim/chive-1.2-mc90.kv')

### 2. ベクトルを利用した単語間の演算
　単語をベクトル化可能としたことにより，単語間で演算ができるようになると考えられる．例えば，英単語「Queen」，「Woman」，「Man」に対して，以下のような方程式を立てる．
$$
\text{Queen} - \text{Woman} = X - \text{Man}
$$
これを$X$について解けば，
$$
X = \text{Queen} - \text{Woman} + \text{Man}
$$
となり，この結果が「King」などの単語となれば理想的である．

In [20]:
X_word = wv.most_similar(positive=['queen', 'man'], negative=['woman'])[0]
print(X_word[0])

king


したがって，予想した通り「King」という単語が得られ，機械学習モデルが単語に関する潜在空間に正しく単語を分類できていること，およびそれを利用して正しく演算ができていることが確認できた．

In [21]:
X_word = wv_jp.most_similar(positive=['王', '女'], negative=['男'])[0]
print(X_word[0])

王女


日本語版に関しても，同様の結果が得られることが確認できた．

### 2. 単語の様々な演算例

#### ・種族変換
$$
\text{Italy} - \text{Pizza} = \text{Japan} - X
$$
$$
X = \text{Pizza} + \text{Japan} -\text{Italy} 
$$
$$
X = \text{Sushi} ?
$$

In [22]:
print(wv.most_similar(positive=['pizza', 'japan'], negative=['italy']))

[('sushi', 0.6840553879737854), ('snack', 0.6649713516235352), ('noodle', 0.6522372364997864), ('toy', 0.6270018219947815), ('oyster', 0.6267036199569702), ('tempura', 0.6250739097595215), ('candy', 0.6205143928527832), ('cracker', 0.6172831654548645), ('sandwich', 0.6168217658996582), ('fried', 0.6153010725975037)]


$$
\text{雷神} - \text{雷} =  X + \text{風}
$$
$$
X = \text{雷神} - \text{雷} + \text{風} 
$$
$$
X = \text{風神} ?
$$

In [23]:
print(wv_jp.most_similar(positive=['雷神', '風'], negative=['雷']))

[('風神', 0.5186702013015747), ('テースト', 0.4441182613372803), ('現代風', 0.4287065267562866), ('和風', 0.42711177468299866), ('アレンジ', 0.42209291458129883), ('今風', 0.41361677646636963), ('異国風', 0.4071023762226105), ('シノワズリー', 0.4044426679611206), ('感じ', 0.40338730812072754), ('オリジナル', 0.396009236574173)]


以上より，2つの単語の組を用いてそれぞれに対応するような単語が連想される場合，word2vecによって上手くそれらを推定することができるとわかる．

#### ・連想
$$
\text{Gravity} + \text{Apple} = X
$$
$$
X = \text{Newton} ?
$$

In [24]:
print(wv.most_similar(positive=['gravity', 'apple']))

[('uses', 0.776378333568573), ('product', 0.7461434602737427), ('using', 0.7373896837234497), ('makes', 0.7290935516357422), ('comes', 0.7158106565475464), ('can', 0.7150611281394958), ('it', 0.7125034332275391), ('device', 0.7122382521629333), ('software', 0.7056861519813538), ('produce', 0.7047755122184753)]


$$
\text{猫} + \text{小説} = X
$$
$$
X = \text{夏目漱石} ?
$$

In [25]:
print(wv_jp.most_similar(positive=['猫', '小説']))

[('短編小説', 0.6366516351699829), ('飼い猫', 0.6274240016937256), ('にゃんこ', 0.6231776475906372), ('猫好き', 0.619087815284729), ('子猫', 0.6177344918251038), ('三毛', 0.6160128712654114), ('漫画', 0.6030644178390503), ('犬', 0.6008184552192688), ('短編', 0.5957843661308289), ('小説家', 0.587696373462677)]


2つの単語の和から，それに関する人物を連想することを試みたが，上手く連想されなかった．これはword2vecに学習された各単語がそれぞれの持つ意味の類似性によって潜在空間に分類されるが，特定の単語の組み合わせの和を求めたところで，新たな単語のベクトルを生み出せるわけではないためであると考えられる．

#### ・中間（平均）
$$
\frac {\text{Hot} + \text{Cold}} {2} = X
$$
$$
X = \text{?}
$$

In [26]:
midpoint = (wv['hot'] + wv['cold']) / 2
print(wv.similar_by_vector(midpoint, topn=5))

[('cold', 0.949198305606842), ('hot', 0.9487224221229553), ('cool', 0.8803013563156128), ('dry', 0.8115673661231995), ('warm', 0.8100968599319458)]


$$
\frac {\text{算数} + \text{国語}} {2} = X
$$
$$
X = \text{?}
$$

In [27]:
midpoint = (wv_jp['算数'] + wv_jp['国語']) / 2
print(wv_jp.similar_by_vector(midpoint, topn=5))

[('算数', 0.9318869709968567), ('国語', 0.9249826073646545), ('数学', 0.7737184762954712), ('教科', 0.7582989931106567), ('現代文', 0.7315290570259094)]


対の意味を持つ単語同士の平均を求めることによって，どのような単語が得られるか調べた．これによって，全く異なる意味の単語が出力されるのではと予想したが，結果はその両方に近い単語が出力された．これは，単語同士（ベクトル）の平均をとることによって，両単語の異なるベクトルの成分は打ち消され，似ている部分のみが残り，結果として両単語に共通する単語が出力されたためであると考えられる．

### 感想  
　単語を上手く潜在空間に分類できていることが面白く，これによって様々な自然言語処理が上手くできていることがわかった．画像や芸術に対してもこのように上手く潜在空間に分類し，より高度な（人間の発想に似たあるいはそれを凌駕する）画像処理が実現すれば面白いと感じた．